In [2]:
import requests
import json
import pandas as pd
from functools import reduce
from pandas.io.json import json_normalize

URL = 'http://real_search:8983/solr/dbnews/select'
T_URL = 'http://search:8983/solr/news/select'
def get_search_result(search_words, from_dt, to_dt):

    num_row = 100
    total = 1
    result_docs = []
    start = 0
    while total > start :
        params = {'debugQuery':'on','indent': 'on', 'q': 'title:"(' + search_words  + ')"~100 AND article:"' + search_words + '" AND datetime:[' + from_dt + 'T00:00:00Z TO ' + to_dt + 'T23:59:59Z]', 'sort': 'datetime desc', 'wt': 'json', 'start':start, 'rows':num_row}
        res = requests.get(URL, params=params)
        response = json.loads(res.text)
        total = response["response"]["numFound"]
        docs = response["response"]["docs"]
        #print("total:%d, start at %d" %(total, start))
        start = start + num_row
        result_docs.append(docs)

    merged_result = reduce(lambda x,y: x+y,result_docs)
    return merged_result


def get_tot_search_result(search_words, from_dt, to_dt):

    num_row = 100
    total = 1
    result_docs = []
    start = 0
    while total > start :
        params = {'debugQuery':'on','indent': 'on', 'q': 'title:"(' + search_words  + ')"~100 AND article:"' + search_words + '" AND datetime:[' + from_dt + 'T00:00:00Z TO ' + to_dt + 'T23:59:59Z]', 'sort': 'datetime desc', 'wt': 'json', 'start':start, 'rows':num_row}
        res = requests.get(T_URL, params=params)
        response = json.loads(res.text)
        total = response["response"]["numFound"]
        docs = response["response"]["docs"]
        #print("total:%d, start at %d" %(total, start))
        start = start + num_row
        result_docs.append(docs)

    merged_result = reduce(lambda x,y: x+y,result_docs)
    return merged_result

In [3]:
week1 = ['2017-03-27','2017-04-02']
week2 = ['2017-04-03','2017-04-09']
week3 = ['2017-04-10','2017-04-16']
week4 = ['2017-04-17','2017-04-23']
week5 = ['2017-04-24','2017-04-30']

week = [
    ['2017-03-27','2017-04-02'], 
    ['2017-04-03','2017-04-09'],
    ['2017-04-10','2017-04-16'],
    ['2017-04-17','2017-04-23'],
    ['2017-04-24','2017-04-30']   
       ]

from_dt = '2017-04-01'
to_dt = '2017-05-09'

In [41]:
df_tot_moon_result = json_normalize(get_tot_search_result('문재인', from_dt, to_dt))
df_tot_ahn_result = json_normalize(get_tot_search_result('안철수',  from_dt, to_dt))
df_tot_hong_result = json_normalize(get_tot_search_result('홍준표', from_dt, to_dt))
df_tot_you_result = json_normalize(get_tot_search_result('유승민',  from_dt, to_dt))
df_tot_sim_result = json_normalize(get_tot_search_result('심상정',  from_dt, to_dt))
    
df_moon_result = json_normalize(get_search_result('문재인', from_dt, to_dt))
df_ahn_result = json_normalize(get_search_result('안철수',  from_dt, to_dt))
df_hong_result = json_normalize(get_search_result('홍준표', from_dt, to_dt))
df_you_result = json_normalize(get_search_result('유승민',  from_dt, to_dt))
df_sim_result = json_normalize(get_search_result('심상정',  from_dt, to_dt))

In [5]:
df_moon_result['tot'] = df_moon_result['n_shares'].fillna(0) + df_moon_result['o_shares'].fillna(0) 
df_ahn_result['tot'] = df_ahn_result['n_shares'].fillna(0) + df_ahn_result['o_shares'].fillna(0) 
df_hong_result['tot'] = df_hong_result['n_shares'].fillna(0) + df_hong_result['o_shares'].fillna(0) 
df_you_result['tot'] = df_you_result['n_shares'].fillna(0) + df_you_result['o_shares'].fillna(0) 
df_sim_result['tot'] = df_sim_result['n_shares'].fillna(0) + df_sim_result['o_shares'].fillna(0) 

In [6]:
df_moon_result['can_no'] = '01'
df_ahn_result['can_no'] = '02'
df_hong_result['can_no'] = '03'
df_you_result['can_no'] = '04'
df_sim_result['can_no'] = '05'

df_all_result=df_moon_result
df_all_result=df_all_result.append([df_ahn_result, df_hong_result, df_you_result, df_sim_result], ignore_index=True)
df_all_result['can_no'].unique()
#df_all_result.tail()

array(['01', '02', '03', '04', '05'], dtype=object)

In [7]:
def extract_sentence(text):
    text = text.replace('\n \n', '\n\n').replace('\t', '')
    sentences = text.split('\n\n')
    return sentences[:-4]

In [20]:
df_all = df_tot_moon_result
#df_ten['article'] = df_ten['article'].str.replace(r'\t|\n|\/\/[a-zA-Z가-힣 \n]+|{}|_flash_removeCallback\(\)|[a-zA-Z0-9]+@yonhapnews.co.kr|\[[a-zA-Z가-힣]\]','')
df_all['article'] = df_all['article'].str.replace(r'\([가-힣=]+\)|\[[a-z0-9가-힣]+\]|\[[a-zA-Z0-9가-힣\(\)\= ]+\]', '')
df_all['article'] = df_all.apply(lambda row: ' '.join(extract_sentence(row['article'])), axis=1)


In [21]:
documents = df_all['article'].tolist()
#pos = lambda d: ['/'.join(p) for p in t.pos(d)]
#texts_ko = [pos(doc) for doc in documents]
#print(texts_ko[0])

In [22]:
from konlpy.tag import Mecab
t=Mecab()
def noun_tagger(text):
    #return [pos[0] for pos in t.pos(text) if pos[1] in ['SL','SH','SN','NN','NNG', 'NNP']]
    return [pos[0] for pos in t.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    #if pos[1] in ['NNP', 'NNG']:

In [23]:
stopwords_kr = []

with open("stopwordlist-kr.txt", "r") as ins:
    for line in ins:
        stopwords_kr.append(line.rstrip('\n'))

http://www.ultravioletanalytics.com/2016/11/18/tf-idf-basics-with-pandas-scikit-learn/

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [32]:
import numpy as np;
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger, stop_words=stopwords_kr, ngram_range=(2,3), min_df=.0025, max_df=.8)
vect_weights = vect.fit_transform(documents)
#sorted(vect.vocabulary_, key=vect.vocabulary_.get, reverse=True)
weights = np.asarray(vect_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vect.get_feature_names(), 'weight': weights})

In [34]:
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
2103,대선 후보,0.035626
2678,문 후보,0.028569
2765,문재인 더불어민주당,0.027848
2362,더불어민주당 대선,0.026203
2365,더불어민주당 대선 후보,0.025995
2767,문재인 더불어민주당 대선,0.025907
2811,문재인 후보,0.019146
6946,후보 측,0.013463
4164,안철수 후보,0.012939
6334,홍 후보,0.012356


In [35]:
weights_df.sort_values(by='weight', ascending=False).head(60)

,term,weight
2103,대선 후보,0.035626
2678,문 후보,0.028569
2765,문재인 더불어민주당,0.027848
2362,더불어민주당 대선,0.026203
2365,더불어민주당 대선 후보,0.025995
2767,문재인 더불어민주당 대선,0.025907
2811,문재인 후보,0.019146
6946,후보 측,0.013463
4164,안철수 후보,0.012939
6334,홍 후보,0.012356


In [43]:
df_ahn = df_tot_ahn_result
#df_ten['article'] = df_ten['article'].str.replace(r'\t|\n|\/\/[a-zA-Z가-힣 \n]+|{}|_flash_removeCallback\(\)|[a-zA-Z0-9]+@yonhapnews.co.kr|\[[a-zA-Z가-힣]\]','')
df_ahn['article'] = df_ahn['article'].str.replace(r'\([가-힣=]+\)|\[[a-z0-9가-힣]+\]|\[[a-zA-Z0-9가-힣\(\)\= ]+\]', '')
df_ahn['article'] = df_ahn.apply(lambda row: ' '.join(extract_sentence(row['article'])), axis=1)
ahn_documents = df_ahn['article'].tolist()

In [46]:
ahn_vect = TfidfVectorizer(tokenizer=noun_tagger, stop_words=stopwords_kr, ngram_range=(2,3), min_df=.0025, max_df=.8)
ahn_vect_weights = ahn_vect.fit_transform(ahn_documents)
#sorted(vect.vocabulary_, key=vect.vocabulary_.get, reverse=True)
ahn_weights = np.asarray(ahn_vect_weights.mean(axis=0)).ravel().tolist()
ahn_weights_df = pd.DataFrame({'term': ahn_vect.get_feature_names(), 'weight': ahn_weights})

In [47]:
ahn_weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
2018,대선 후보,0.038103
3856,안철수 국민의당,0.030226
1431,국민의당 대선,0.028952
1434,국민의당 대선 후보,0.028694
3857,안철수 국민의당 대선,0.028485
3789,안 후보,0.023374
3912,안철수 후보,0.023172
1455,국민의당 안철수,0.015594
2540,문 후보,0.014142
4017,여론 조사,0.012528


In [113]:
t.pos("최순실")

[('최순실', 'NNP')]

In [112]:
from gensim.summarization import keywords
from gensim.summarization import summarize
text = "\'비선실세\' 최순실씨(61)가 법정에서 검찰 측에 \'박근혜 전 대통령에 대한 예의를 지키라\'며 지적했다가 재판부의 제지를 당했다. 최씨는 박 전 대통령이 SK그룹으로부터 현안에 도움을 주는 대가로 뇌물을 받으려 했다는 혐의를 부인했다. 서울중앙지법 형사합의22부(부장판사 김세윤) 심리로 15일 열린 박근혜 전 대통령(65) 등에 대한 재판에서 최씨는 발언권을 얻어 검찰 측 강일민 검사에게 \"검사님, 저희가 모신 대통령이니까 예의를 지켰으면 좋겠다\"고 말했다. 앞서 강 검사는 변호인 측이 증인에 대한 반대신문에서 핵심을 호도하고 있다고 큰 목소리로 지적했다. 강 검사는 \"(변호인의) 현란한 유도신문이 나왔다\"며 \"박 전 대통령과 최태원 SK그룹 회장의 독대에서 SK텔레콤의 CJ헬로비전 인수합병과, 면세점 재취득, 최재원 석방 등 SK 측의 현안이 언급된 게 핵심\"이라고 말했다. 최씨의 \"예의를 지키라\"는 지적에 재판부는 \"증인에게 질문만 하라\"며 제지했다. 그는 이날 증인으로 출석한 이형희 SK브로드밴드 사장을 향해 \"흥분한 검사님(강 검사)은 SK에 현안이 많다고 했지만 그런 SK의 요구를 (박 전 대통령이) 들어 준 게 없지 않느냐\"고 주장했다. 검찰은 독대에서 박 전 대통령이 최 회장에게 미르·K스포츠재단 등에 대한 지원을 요구하고, 최 회장은 그 대가로 SK의 각종 현안에 대한 도움을 요청했다고 본다. 최씨의 질문은 이런 검찰의 시각이 잘못됐다는 취지다. 최씨는 \"제 생각에 (지원금과) SK의 민원사항을 연결하는 건 무리\"라며 \"검찰 측은 제가 사익을 추구하기 위해 박 전 대통령이 (SK의 부탁을 들어줬다며) 공범으로 몰고간 것\"이라고 말했다. 박 전 대통령은 \'증인에게 물어볼 것이 있느냐\'는 재판부의 질문에 고개를 절레절레 저었다. 지난달 23일 첫 재판부터 이날까지 단 한번도 증인에게 질문하지 않은 박 전 대통령은 아무런 대답 없이 고개만 젓는 모습을 자주 보였다. 이날 재판에선 최 회장이 자신에게 혼외자가 있다는 사실을 언론에 보도되기 전에 박 전 대통령에게 사전에 알렸다는 정황이 제시되기도 했다. 이 사장은 \'2015년 12월 최 회장이 안종범 전 청와대 정책조정수석에게 혼외자 있다는 걸 알리면서 이에 대해 미리 대통령에 전달해 양해를 구하라고 부탁한 사실이 있느냐\'는 질문에 \"그렇다\"고 답했다. 그는 \'대통령이 경제를 살리기 위해 사면해줬는데 혼외자 문제로 부정적인 여론이 형성돼 양해를 구하기 위한 목적이었느냐\'는 질문에 동의하면서 \"대통령에게 사전에 알리는 차원이었다\"고 설명했다. 이 사장은 \"안 전 수석이 이를 박 전 대통령에게 전달한 것으로 안다\"며 \"(박 전 대통령은) 알았다는 반응이었고 약간 조심스러워했다(고 들었다)\"고 말했다."
print('Keywords:')
print(keywords(text)) 
print('Summary:')
print(summarize(text, ratio=0.01))

Keywords:
증인에게
sk의
예의를
최씨는
양해를
혼외자
질문에
현안이
검사님
최씨의
대통령은
독대에서
김세윤
대통령에게 사전에
재판부의
있다는
대통령이
핵심을 호도하고
sk브로드밴드
많다고
검사에게
출석한 이형희
검사는
유도신문이
재취득
서울중앙지법
Summary:



In [87]:
text1="애플 실적을 알려줘"

In [88]:
print(t.pos(text1))

[('애플', 'NNP'), ('실적', 'NNG'), ('을', 'JKO'), ('알려줘', 'VV+EC+VX+EC')]


In [107]:
print(t.pos("북한정권은 도저히 인간이라고 볼 수없는 극악무도한 패륜집단이다. 전 세계가 나서서 김정은과 그 추종자들을 처단하여 인류역사의 오점을 바로잡아야 한다. 이런 집단과 조건없이 대화하고 6.15, 10.4합의를 이행하겠다는 문대통령은 잘못된 선택을 한 것이다. 한민족이 세계로부터 외면당하고 외톨이가 될 것이다."))

[('북한', 'NNP'), ('정권', 'NNG'), ('은', 'JX'), ('도저히', 'MAG'), ('인간', 'NNG'), ('이', 'VCP'), ('라고', 'EC'), ('볼', 'VX+ETM'), ('수', 'NNB'), ('없', 'VA'), ('는', 'ETM'), ('극악무도', 'NNG'), ('한', 'XSA+ETM'), ('패륜', 'NNG'), ('집단', 'NNG'), ('이', 'VCP'), ('다', 'EF'), ('.', 'SF'), ('전', 'MM'), ('세계', 'NNG'), ('가', 'JKS'), ('나서', 'VV'), ('서', 'EC'), ('김정은', 'NNP'), ('과', 'JC'), ('그', 'MM'), ('추종', 'NNG'), ('자', 'XSN'), ('들', 'XSN'), ('을', 'JKO'), ('처단', 'NNG'), ('하', 'XSV'), ('여', 'EC'), ('인류', 'NNG'), ('역사', 'NNG'), ('의', 'JKG'), ('오점', 'NNG'), ('을', 'JKO'), ('바로잡', 'VV'), ('아야', 'EC'), ('한다', 'VX+EF'), ('.', 'SF'), ('이런', 'MM'), ('집단', 'NNG'), ('과', 'JC'), ('조건', 'NNG'), ('없이', 'MAG'), ('대화', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('6', 'SN'), ('.', 'SY'), ('15', 'SN'), (',', 'SC'), ('10', 'SN'), ('.', 'SY'), ('4', 'SN'), ('합의', 'NNG'), ('를', 'JKO'), ('이행', 'NNG'), ('하', 'XSV'), ('겠', 'EP'), ('다는', 'ETM'), ('문', 'NNG'), ('대통령', 'NNG'), ('은', 'JX'), ('잘못', 'MAG'), ('된', 'VV+ETM'), ('선택', 'NNG'), ('을', 'JK

In [106]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
text = "지난 주 애플 실적을 알려줘"
token=t.pos(text)
bigrams=ngrams(token,2)
for grams in bigrams :
    print(grams)

(('지난', 'VV+ETM'), ('주', 'NNP'))
(('주', 'NNP'), ('애플', 'NNP'))
(('애플', 'NNP'), ('실적', 'NNG'))
(('실적', 'NNG'), ('을', 'JKO'))
(('을', 'JKO'), ('알려줘', 'VV+EC+VX+EC'))


http://scikit-learn.org/stable/auto_examples/text/document_clustering.html#sphx-glr-auto-examples-text-document-clustering-py

https://www.youtube.com/watch?v=znfy3T9OiAQ